<a href="https://colab.research.google.com/github/JuneWayne/RAG-Based-Chatbot-Demo/blob/main/Colab_Inline_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Based chabot demo

## Simple chatbot with RetrievalQA

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf
!pip install -U langchain-community
import os
import tempfile

import panel as pn
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

loader = PyPDFLoader('Jonathan_Pau_Interview_Notes_Detailed.pdf')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

retriever = docsearch.as_retriever(
    search_type='similarity', search_kwargs={'k':2} # k=2 finds the top 2 most relevant text chunk
)     # similarity search retrieves text chunk vectors that ar emost similar to the question

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type='map_reduce', #
    retriever=retriever,
    return_source_documents=True,
    verbose=False
)

In [ ]:
qa('what are the functions of DE')

In [ ]:
qa('what is newton''s third law')

## Colab Inline interface with RAG application

In [ ]:
import os
import tempfile
import panel as pn

pn.extension("colab")

pdf_input = pn.widgets.FileInput(accept=".pdf", value=None, height=50)

key_input = pn.widgets.PasswordInput(
    name="OpenAI Key",
    placeholder="Please paste in your OpenAI API key",
)

k_slider = pn.widgets.IntSlider(
    name="Number of Relevant Chunks", start=1, end=5, step=1, value=2
)

chain_select = pn.widgets.RadioButtonGroup(
    name="Chain Type", options=["stuff", "map_reduce", "refine", "map_rerank"], value="stuff"
)

chat_input = pn.widgets.TextInput(placeholder="Ask a question about the uploaded PDF...")
chat_output = pn.pane.Markdown("## The response will appear here.", sizing_mode="stretch_width")

def initialize_chain():
    if not key_input.value:
        chat_output.object = "Please provide your OpenAI API Key!"
        return None

    os.environ["OPENAI_API_KEY"] = key_input.value

    if not pdf_input.value:
        chat_output.object = "Please upload a PDF file."
        return None

    if not chain_select.value:
        chat_output.object = "Please select a chain type."
        return None

    chat_input.placeholder = "Ask questions here!"
    with tempfile.NamedTemporaryFile("wb", delete=False) as f:
        f.write(pdf_input.value)
        file_name = f.name

    loader = PyPDFLoader(file_name)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(texts, embeddings)

    retriever = db.as_retriever(
        search_type="similarity", search_kwargs={"k": k_slider.value}
    )

    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(),
        chain_type=chain_select.value,
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
    )

    chat_output.object = "Thinking about this..."
    return qa

def on_submit(event=None):
    qa_chain = initialize_chain()
    if qa_chain:
        question = chat_input.value
        if question:
            response = qa_chain(question)
            chat_output.object = (
                "<div style='font-size:20px;'>"
                f"Answer:\n{response['result']}\n\n"
                #f"### Source Documents:\n{[doc.metadata.get('source', 'unknown') for doc in response['source_documents']]}"
                "</div>"
            )

chat_input.param.watch(on_submit, 'value')

dashboard = pn.Column(
    "# Mini OpenAI RAG Based PDF Ingesting ChatBot",
    pdf_input,
    key_input,
    k_slider,
    chain_select,
    chat_input,
    chat_output,
    sizing_mode="stretch_width"
)

dashboard
